# Conversational Interface - Chatbot with Claude LLM

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude as our FM for building the chatbot.

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In [14]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

  Obtaining dependency information for boto3>1.28.57 from https://files.pythonhosted.org/packages/88/a3/e63a11c957c1e2a901d13f3ddc2acba2d0db8a7bcbac2e3f5b06aab0c6de/boto3-1.28.59-py3-none-any.whl.metadata
  Using cached boto3-1.28.59-py3-none-any.whl.metadata (6.7 kB)
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/ad/45/3a4ee2a246ec9e38b1b955bb27f37a6f256dbfa2749bc14445e919df067b/awscli-1.29.59-py3-none-any.whl.metadata
  Using cached awscli-1.29.59-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/90/4c/864b051e7227178fd90f663c667d2483503ca129ed30b26860bcd1d3ea71/botocore-1.31.59-py3-none-any.whl.metadata
  Using cached botocore-1.31.59-py3-none-any.whl.metadata (6.0 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8

In this notebook, we'll also need some extra dependencies:

- [FAISS](https://github.com/facebookresearch/faiss), to store vector embeddings
- [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), for interactive UI widgets in the notebook
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files

In [20]:
%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.304 "pypdf>=3.8,<4"
%pip install --upgrade sqlalchemy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::195364414018:role/Crossaccountbedrock"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
  Using role: arn:aws:iam::195364414018:role/Crossaccountbedrock ... successful!
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


## Chatbot with Context 
In this use case we will ask the Chatbot to answer question from some external corpus it has likely never seen before. To do this we apply a pattern called RAG (Retrieval Augmented Generation): the idea is to index the corpus in chunks, then look up which sections of the corpus might be relevant to provide an answer by using semantic similarity between the chunks and the question. Finally the most relevant chunks are aggregated and passed as context to the ConversationChain, similar to providing a history.

We will take a csv file and use **Titan Embeddings Model** to create vectors for each line of the csv. This vector is then stored in FAISS, an open source library providing an in-memory vector datastore. When the chatbot is asked a question, we query FAISS with the question and retrieve the text which is semantically closest. This will be our answer. 

#### Titan embeddings Model

Embeddings are a way to represent words, phrases or any other discrete items as vectors in a continuous vector space. This allows machine learning models to perform mathematical operations on these representations and capture semantic relationships between them.

Embeddings are for example used for the RAG [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/) 


In [22]:
from langchain.embeddings import BedrockEmbeddings
# anthropic.claude-v2
# amazon.titan-embed-g1-text-02
br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

#### FAISS as VectorStore

In order to be able to use embeddings for search, we need a store that can efficiently perform vector similarity searches. In this notebook we use FAISS, which is an in memory store. For permanently store vectors, one can use pgVector, Pinecone or Chroma.

The langchain VectorStore API's are available [here](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

To know more about the FAISS vector store please refer to this [document](https://arxiv.org/pdf/1702.08734.pdf).

In [23]:
from langchain.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

loader = PyPDFLoader("policy_certifcate_multiple_vehicle.pdf")
pages = loader.load()

chunk_size = 1000
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=100,
    length_function = len,
)

docs, metadata = [], []

for i in range(len(pages)):
    print(f"Spliting the content with length", len(pages[i].page_content))
    splits = text_splitter.split_text(pages[i].page_content)
    docs.extend(splits)
    metadata.extend([{"source": pages[i].metadata["source"]}] * len(splits))

vectorstore_faiss_aws = FAISS.from_texts(
    docs,
    br_embeddings,
    metadatas=metadata,
)

print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}")
# with open("vectorstore_faiss_aws.pkl", "wb") as f:
#     pickle.dump(vectorstore_faiss_aws, f)
    


Spliting the content with length 2579
Spliting the content with length 85
Spliting the content with length 1269
Spliting the content with length 3463
Spliting the content with length 85
Spliting the content with length 1295
Spliting the content with length 85
Spliting the content with length 2567
Spliting the content with length 708
Spliting the content with length 2789
Spliting the content with length 2121
Spliting the content with length 819
Spliting the content with length 85
Spliting the content with length 2850
Spliting the content with length 2124
Spliting the content with length 2851
Spliting the content with length 2124
vectorstore_faiss_aws: number of elements in the index=41


#### Semantic search

We can use a Wrapper class provided by LangChain to query the vector data base store and return to us the relevant documents. Behind the scenes this is only going to run a RetrievalQA chain.

In [24]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=cl_llm))

ImportError: cannot import name 'URL' from 'sqlalchemy' (/opt/conda/lib/python3.10/site-packages/sqlalchemy/__init__.py)

Let's see how the semantic search works:
1. First we calculate the embeddings vector for the query, and
2. then we use this vector to do a similarity search on the store

In [ ]:
v = br_embeddings.embed_query("R in SageMaker")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=4)
for r in results:
    print_ww(r.page_content)
    print('----')

#### Memory
In any chatbot we will need a QA Chain with various options which are customized by the use case. But in a chatbot we will always need to keep the history of the conversation so the model can take it into consideration to provide the answer. In this example we use the [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) from LangChain, together with a ConversationBufferMemory to keep the history of the conversation.

Source: https://python.langchain.com/docs/modules/chains/popular/chat_vector_db

Set `verbose` to `True` to see all the what is going on behind the scenes.

In [ ]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

#### Parameters used for ConversationRetrievalChain
* **retriever**: We used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types you can choose: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object where it selects text chunk vectors that are most similar to the question vector.

* **memory**: Memory Chain to store the history 

* **condense_question_prompt**: Given a question from the user, we use the previous conversation and that question to make up a standalone question

* **chain_type**: If the chat history is long and doesn't fit the context you use this parameter and the options are `stuff`, `refine`, `map_reduce`, `map-rerank`

If the question asked is outside the scope of context, then the model will reply it doesn't know the answer

**Note**: if you are curious how the chain works, uncomment the `verbose=True` line.

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message("You will be acting as a Insurance Assistant for Aviva an insurance company. Your goal is to provide correct and valid answer for user queries.")
memory.chat_memory.add_ai_message("Ok, I am a insurance assistant for aviva and give proper answer for customer queries.")

cl_llm = Bedrock(model_id="anthropic.claude-v2",client=boto3_bedrock)

qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

Let's chat! ask the chatbot some questions about SageMaker, like:
1. What is SageMaker?
2. What is canvas?

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Your mileage might vary, but after 2 or 3 questions you will start to get some weird answers. In some cases, even in other languages.
This is happening for the same reasons outlined at the beginning of this notebook: the default langchain prompts are not optimal for Claude. 
In the following cell we are going to set two new prompts: one for the question rephrasing, and one to get the answer from that rephrased question.

# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

Answer only with the new question.


Human: How would you ask the question considering the previous conversation: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provides longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

Let's start another chat. Feel free to ask the following questions:

1. What is SageMaker?
2. what is canvas?

chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

#### Do some prompt engineering

You can "tune" your prompt to get more or less verbose answers. For example, try to change the number of sentences, or remove that instruction all-together. You might also need to change the number of `max_tokens_to_sample` (eg 1000 or 2000) to get the full answer.

### In this demo we used Claude LLM to create conversational interface with following patterns:

1. Chatbot (Basic - without context)

2. Chatbot using prompt template(Langchain)

3. Chatbot with personas

4. Chatbot with context